# Point cloud registration with RMC-Open3D

In [1]:
from rmc_open3d import utils
from rmc_open3d.icp import ICP

import logging
import open3d as o3d
import numpy as np
import json

## Load camera parameters from BlenderProc BopWriter

In [2]:
scene_id = 30
path_to_scene_camera_json = "tests/test_data/bop_data/obj_of_interest/train_pbr/000000/scene_camera.json"
path_to_camera_json = "tests/test_data/bop_data/obj_of_interest/camera.json"
output_path = "tests/test_data"

camera_parameters = utils.get_camera_parameters_from_blenderproc_bopwriter(scene_id,
                                                                           path_to_scene_camera_json,
                                                                           path_to_camera_json,
                                                                           output_path)

## Load target RGB-D image

In [8]:
camera_parameters = o3d.io.read_pinhole_camera_parameters("tests/test_data/camera_parameters.json")
color = f"tests/test_data/bop_data/obj_of_interest/train_pbr/000000/rgb/000030.png"
depth = f"tests/test_data/bop_data/obj_of_interest/train_pbr/000000/depth/000030.png"

target_rgbd = utils.convert_rgbd_image_to_point_cloud(rgbd_image_or_path=[color, depth],
                                                      camera_intrinsic=camera_parameters.intrinsic,
                                                      camera_extrinsic=camera_parameters.extrinsic,
                                                      depth_scale=1000.0,
                                                      depth_trunc=2.0)

o3d.visualization.draw_geometries([target_rgbd])

## ICP on source and target point cloud

In [5]:
# Load ground truth pose
with open("tests/test_data/ground_truth_pose.json") as f:
    gt_pose_data = json.load(f)
rotation = gt_pose_data["rotation_quaternion"]
translation = gt_pose_data["translation_xyz"]
gt_pose = utils.get_transformation_matrix_from_quaternion(rotation_wxyz=rotation,
                                                          translation_xyz=translation)

source_path = "tests/test_data/suzanne.ply"
source_pcd = utils.eval_data(data=source_path,
                             number_of_points=5000)

target_path = "tests/test_data/chair_with_suzanne.ply"
target_pcd = utils.eval_data(data=target_path,
                             number_of_points=100000)

icp = ICP()
init = np.eye(4)
init[:3, 3] = [0.1, 0.1, 0.4]
result = icp.run(source=source_pcd,
                 target=target_pcd,
                 init=init,
                 max_iteration=100,
                 visualize=True)

print(np.linalg.norm(gt_pose - result.transformation))

0.008391136238148206


## ICP on source point cloud and target depth image

In [6]:
result = icp.run(source=source_pcd,
                 target=target_rgbd,
                 init=init,
                 max_iteration=100,
                 visualize=True)

print(np.linalg.norm(gt_pose - result.transformation))

0.06983415464983501
